# ⏱️ Building a Word-Counter Video with VideoDB

<a href="https://colab.research.google.com/github/video-db/videodb-cookbook/blob/main/examples/Keyword_Search_Counter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Introduction
With an endless stream of new video content on our feeds, engaging the audience with dynamic visual elements can make educational and promotional videos much more impactful. VideoDB's suite of features allows you to enhance videos with programmatic editing.

In this tutorial, we'll explore how to create a video that visually counts and displays instances of a specified word as it's spoken. We'll use VideoDB’s [Keyword Search ](https://docs.videodb.io)to index spoken words, and then apply audio and [text overlays](https://docs.videodb.io) to show a counter updating in real-time with synchronized audio cues.

## Setup
---


### 📦  Installing packages

In [ ]:
%pip install -q videodb

### 🔑 API Keys
Before proceeding, ensure access to [VideoDB](https://videodb.io) API key.

> Get your API key from [VideoDB Console](https://console.videodb.io). ( Free for first 50 uploads, **No credit card required** ) 🎉

In [ ]:
import videodb
import os
from getpass import getpass

# Prompt user for API key securely
api_key = getpass("Please enter your VideoDB API Key: ")
os.environ["VIDEO_DB_API_KEY"] = api_key

Please enter your VideoDB API Key: ··········


## Steps
---

### 🔗  Step 1: Connect to VideoDB

In [ ]:
import videodb

conn = videodb.connect()
coll = conn.get_collection()

### 🎥 Step 2: Upload Video
Upload and play the video to ensure it's correctly loaded. We’ll be using [this video](https://www.youtube.com/watch?v=Js4rTM2Z1Eg) for the purpose of this tutorial.

In [ ]:
video = coll.upload(url="https://www.youtube.com/watch?v=Js4rTM2Z1Eg")
print("Video ID: ", video.id)

Video ID:  m-z-019b9261-7d0d-7c51-99ee-321302b08e79


### Lets see the original video

In [ ]:
video.play()

### 📝 Step 3:  Indexing Spoken Words
Index the video to identify and timestamp all spoken words.

In [ ]:
video.index_spoken_words()

100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:31<00:00,  3.22it/s]


### 🔍  Step 4:  Keyword Search

Search within the video for the keyword ("education" in this example), and note each occurrence.


In [ ]:
from videodb import SearchType

result = video.search(query='education', search_type=SearchType.keyword)

result.get_shots()

[Shot(video_id=m-z-019b9261-7d0d-7c51-99ee-321302b08e79, video_title=EDUCATION IS THE MOST IMPORTANT THING|| Jack Ma Motivational speech ||Motivated soul|Motivation💪💪, start=7.76, end=8.08, text=education, search_score=1, stream_url=None, player_url=None),
 Shot(video_id=m-z-019b9261-7d0d-7c51-99ee-321302b08e79, video_title=EDUCATION IS THE MOST IMPORTANT THING|| Jack Ma Motivational speech ||Motivated soul|Motivation💪💪, start=14.88, end=15.28, text=education, search_score=1, stream_url=None, player_url=None),
 Shot(video_id=m-z-019b9261-7d0d-7c51-99ee-321302b08e79, video_title=EDUCATION IS THE MOST IMPORTANT THING|| Jack Ma Motivational speech ||Motivated soul|Motivation💪💪, start=22.24, end=22.56, text=education, search_score=1, stream_url=None, player_url=None),
 Shot(video_id=m-z-019b9261-7d0d-7c51-99ee-321302b08e79, video_title=EDUCATION IS THE MOST IMPORTANT THING|| Jack Ma Motivational speech ||Motivated soul|Motivation💪💪, start=57.29, end=57.57, text=education, search_score=1, s

### 🎼  Step 5:  Setup Timeline

Initialize the timeline and prepare an audio asset to use for each word occurrence.

In [ ]:
from videodb import MediaType
# Upload the twink sound effect
audio = conn.upload(url="https://github.com/video-db/videodb-cookbook-assets/raw/main/audios/twink.mp3", media_type=MediaType.audio)
print("Audio ID: ", audio.id)

Audio ID:  a-z-019b9265-99ff-7cf3-b72e-f41e81437d19


### Initializing Timeline

In [ ]:
from videodb.editor import Timeline, Track, Clip, AudioAsset, VideoAsset, TextAsset
from videodb.editor import Font, Background, Alignment, HorizontalAlignment, VerticalAlignment, Position, Offset
from videodb import MediaType

timeline = Timeline(conn)

### 💬 Step 6:  Overlay Text and Audio

Add text and audio overlays at each instance where the word is spoken using the `Track` and `Clip` pattern.

>
>Note: Adding the ‘padding’ is an optional step. It helps in adding a little more context to the exact instance identified, thus resulting in a better compiled output.
>

In [ ]:
video_duration = min(300, int(video.length))  # First 5 minutes only
audio_offset = 1  # Delay audio/text update by 1 second for better sync

# Create timeline and tracks
timeline = Timeline(conn)
video_track = Track()
text_track = Track()
audio_track = Track()

# Add video clip (first 5 minutes)
video_clip = Clip(
    asset=VideoAsset(id=video.id, start=0),
    duration=video_duration
)
video_track.add_clip(0, video_clip)

# Filter shots within our duration
shots_in_range = [s for s in result.shots if int(s.start) + audio_offset < video_duration]

# Add text overlays that update at each word occurrence
for i, shot in enumerate(shots_in_range):
    trigger_time = int(shot.start) + audio_offset

    # Initial "Count-0" from start until first word
    if i == 0 and trigger_time > 0:
        text_asset = TextAsset(
            text="Count-0",
            font=Font(family="Do Hyeon", size=72, color="#000100"),
            background=Background(color="#F702A4", opacity=1.0),
            alignment=Alignment(horizontal=HorizontalAlignment.right, vertical=VerticalAlignment.top),
        )
        text_clip = Clip(asset=text_asset, duration=trigger_time, position=Position.top_right, offset=Offset(x=-0.05, y=0.05))
        text_track.add_clip(0, text_clip)

    # Duration until next word or end of video
    if i + 1 < len(shots_in_range):
        next_trigger = int(shots_in_range[i + 1].start) + audio_offset
    else:
        next_trigger = video_duration

    text_dur = next_trigger - trigger_time

    # Text overlay with updated count
    text_asset = TextAsset(
        text=f"Count-{i + 1}",
        font=Font(family="Do Hyeon", size=72, color="#000100"),
        background=Background(color="#F702A4", opacity=1.0),
        alignment=Alignment(horizontal=HorizontalAlignment.right, vertical=VerticalAlignment.top),
    )
    text_clip = Clip(asset=text_asset, duration=text_dur, position=Position.top_right, offset=Offset(x=-0.05, y=0.05))
    text_track.add_clip(trigger_time, text_clip)

    # Audio cue at same trigger time
    if trigger_time < video_duration - 2:
        audio_clip = Clip(asset=AudioAsset(id=audio.id), duration=1)
        audio_track.add_clip(trigger_time, audio_clip)

# Add all tracks to timeline
timeline.add_track(video_track)
timeline.add_track(text_track)
timeline.add_track(audio_track)

### ⚡️ Step 7: Generate and Play the Stream

Finally, generate a streaming URL for your edited video and play it.

In [ ]:
from videodb import play_stream

stream_url = timeline.generate_stream()
play_stream(stream_url)


## Conclusion
---

This tutorial showcases VideoDB's capabilities to create a video that programmatically counts and displays the frequency of a specific keyword spoken throughout the video. This method can be adapted for various applications where dynamic text overlays add significant value to video content.

### Tips and Tricks

* Use different text styles and positions based on your video's theme.
* Add background sounds or effects to enhance the viewer's experience.